<a href="https://colab.research.google.com/github/jaawad81/deeplearningCOM762/blob/main/Week_3_Deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U scikit-learn tensorflow

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error
from sklearn.datasets import load_breast_cancer, load_iris, fetch_california_housing

from sklearn.neural_network import MLPClassifier, MLPRegressor

import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Make results reproducible (important for teaching demos)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.20.0


In [ ]:
# ============================================================
# SECTION A: Scikit-learn MLP (Binary Classification)
# Dataset: Breast Cancer (binary: malignant vs benign)
# ============================================================

# ----------------------------
# A1) Load Data
# ----------------------------
data = load_breast_cancer()
X = data.data                  # Features (shape: n_samples x n_features)
y = data.target                # Labels (0/1)

print("\n[Binary - Breast Cancer]")
print("X shape:", X.shape, "| y shape:", y.shape)
print("Classes:", data.target_names)


[Binary - Breast Cancer]
X shape: (569, 30) | y shape: (569,)
Classes: ['malignant' 'benign']


In [ ]:
# ----------------------------
# A2) Split Data (Train/Test)
# ----------------------------
# test_size=0.2 means 20% used for testing (holdout set).
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# ----------------------------
# A3) Standardize Features
# ----------------------------
# Neural networks often work better when inputs are scaled.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


In [ ]:
# ----------------------------
# A4) Build + Train Scikit-learn MLPClassifier
# ----------------------------
# hidden_layer_sizes=(30, 15) means 2 hidden layers: 30 neurons then 15 neurons.
# max_iter controls training iterations for scikit MLP.
clf = MLPClassifier(
    hidden_layer_sizes=(30, 15),
    activation="relu",
    solver="adam",
    alpha=1e-4,               # L2 regularization term
    max_iter=500,
    random_state=42
)

clf.fit(X_train_scaled, y_train)

,"hidden_layer_sizes hidden_layer_sizes: array-like of shape(n_layers - 2,), default=(100,)The ith element represents the number of neurons in the ithhidden layer.","(30, ...)"
,"activation activation: {'identity', 'logistic', 'tanh', 'relu'}, default='relu'Activation function for the hidden layer.- 'identity', no-op activation, useful to implement linear bottleneck, returns f(x) = x- 'logistic', the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).- 'tanh', the hyperbolic tan function, returns f(x) = tanh(x).- 'relu', the rectified linear unit function, returns f(x) = max(0, x)",'relu'
,"solver solver: {'lbfgs', 'sgd', 'adam'}, default='adam'The solver for weight optimization.- 'lbfgs' is an optimizer in the family of quasi-Newton methods.- 'sgd' refers to stochastic gradient descent.- 'adam' refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy BaFor a comparison between Adam optimizer and SGD, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_training_curves.py`.Note: The default solver 'adam' works pretty well on relativelylarge datasets (with thousands of training samples or more) in terms ofboth training time and validation score.For small datasets, however, 'lbfgs' can converge faster and performbetter.",'adam'
,"alpha alpha: float, default=0.0001Strength of the L2 regularization term. The L2 regularization termis divided by the sample size when added to the loss.For an example usage and visualization of varying regularization, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_alpha.py`.",0.0001
,"batch_size batch_size: int, default='auto'Size of minibatches for stochastic optimizers.If the solver is 'lbfgs', the classifier will not use minibatch.When set to ""auto"", `batch_size=min(200, n_samples)`.",'auto'
,"learning_rate learning_rate: {'constant', 'invscaling', 'adaptive'}, default='constant'Learning rate schedule for weight updates.- 'constant' is a constant learning rate given by 'learning_rate_init'.- 'invscaling' gradually decreases the learning rate at each time step 't' using an inverse scaling exponent of 'power_t'. effective_learning_rate = learning_rate_init / pow(t, power_t)- 'adaptive' keeps the learning rate constant to 'learning_rate_init' as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol if 'early_stopping' is on, the current learning rate is divided by 5.Only used when ``solver='sgd'``.",'constant'
,"learning_rate_init learning_rate_init: float, default=0.001The initial learning rate used. It controls the step-sizein updating the weights. Only used when solver='sgd' or 'adam'.",0.001
,"power_t power_t: float, default=0.5The exponent for inverse scaling learning rate.It is used in updating effective learning rate when the learning_rateis set to 'invscaling'. Only used when solver='sgd'.",0.5
,"max_iter max_iter: int, default=200Maximum number of iterations. The solver iterates until convergence(determined by 'tol') or this number of iterations. For stochasticsolvers ('sgd', 'adam'), note that this determines the number of epochs(how many times each data point will be used), not the number ofgradient steps.",500
,"shuffle shuffle: bool, default=TrueWhether to shuffle samples in each iteration. Only used whensolver='sgd' or 'adam'.",True
,"random_state random_state: int, RandomState instance, default=NoneDetermines random number generation for weights and biasinitialization, train-test split if early stopping is used, and batchsampling when solver='sgd' or 'adam'.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",42


In [ ]:
# ----------------------------
# A5) Evaluate
# ----------------------------
y_pred = clf.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)

print("\nScikit-learn MLPClassifier Accuracy:", round(acc, 4))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=data.target_names))


Scikit-learn MLPClassifier Accuracy: 0.9561

Confusion Matrix:
 [[41  1]
 [ 4 68]]

Classification Report:
               precision    recall  f1-score   support

   malignant       0.91      0.98      0.94        42
      benign       0.99      0.94      0.96        72

    accuracy                           0.96       114
   macro avg       0.95      0.96      0.95       114
weighted avg       0.96      0.96      0.96       114



In [ ]:
# ----------------------------
# A6) Predict Probabilities
# ----------------------------
# predict_proba returns class probabilities: P(class=0), P(class=1)
probs = clf.predict_proba(X_test_scaled[:5])
print("\nFirst 5 probability predictions (rows):\n", probs)



First 5 probability predictions (rows):
 [[1.00000000e+00 1.15908235e-10]
 [1.34192839e-08 9.99999987e-01]
 [9.99880832e-01 1.19167575e-04]
 [6.23656553e-01 3.76343447e-01]
 [1.00000000e+00 6.97159883e-13]]


In [ ]:
# ============================================================
# SECTION B: Keras Sequential (Binary Classification)
# Same dataset: Breast Cancer
# Lifecycle: Define -> Compile -> Fit -> Evaluate -> Predict
# ============================================================

print("\n[Keras - Binary Classification]")

# ----------------------------
# B1) Define the Model
# ----------------------------
# For binary classification:
# - output layer has 1 neuron with sigmoid (gives value between 0 and 1)
keras_bin = Sequential([
    Dense(32, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")
])



[Keras - Binary Classification]


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# ----------------------------
# B2) Compile the Model
# ----------------------------
# Loss for binary classification: binary_crossentropy
# Optimizer: adam (good default)
# Metric: accuracy
keras_bin.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
# ----------------------------
# B3) Train (Fit)
# ----------------------------
# EarlyStopping stops training if validation loss doesn't improve,
# which prevents overfitting and speeds up demos.
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = keras_bin.fit(
    X_train_scaled, y_train,
    validation_split=0.2,   # 20% of training set used as validation
    epochs=200,
    batch_size=32,
    verbose=0,
    callbacks=[early_stop]
)

print("Training finished. Best val_loss:", np.min(history.history["val_loss"]))

Training finished. Best val_loss: 0.013884516432881355


In [ ]:
# ----------------------------
# B4) Evaluate
# ----------------------------
loss, acc = keras_bin.evaluate(X_test_scaled, y_test, verbose=0)
print("Keras Binary Test Loss:", round(loss, 4))
print("Keras Binary Test Accuracy:", round(acc, 4))

Keras Binary Test Loss: 0.1227
Keras Binary Test Accuracy: 0.9737


In [ ]:
# ----------------------------
# B5) Predict
# ----------------------------
# Predict returns probabilities; convert to class with threshold 0.5
pred_probs = keras_bin.predict(X_test_scaled[:10], verbose=0).reshape(-1)
pred_classes = (pred_probs >= 0.5).astype(int)

print("\nFirst 10 predicted probabilities:", np.round(pred_probs, 4))
print("First 10 predicted classes:", pred_classes)
print("First 10 true classes     :", y_test[:10])



First 10 predicted probabilities: [0.     1.     0.     0.041  0.     0.9999 1.     0.     0.     0.    ]
First 10 predicted classes: [0 1 0 0 0 1 1 0 0 0]
First 10 true classes     : [0 1 0 1 0 1 1 0 0 0]


In [ ]:
# ----------------------------
# B6) Save + Load Model (Optional but useful for teaching)
# ----------------------------
# In real projects you often save the trained model and load later for predictions.
keras_bin.save("keras_binary_model.keras")
loaded_bin = tf.keras.models.load_model("keras_binary_model.keras")

# Quick check
loaded_preds = loaded_bin.predict(X_test_scaled[:3], verbose=0).reshape(-1)
print("\nLoaded model predictions (first 3):", np.round(loaded_preds, 4))




Loaded model predictions (first 3): [0. 1. 0.]


In [ ]:
# ============================================================
# SECTION C: Keras Sequential (Multiclass Classification)
# Dataset: Iris (3 classes)
# ============================================================

print("\n[Keras - Multiclass Classification (Iris)]")

# ----------------------------
# C1) Load Data
# ----------------------------
iris = load_iris()
X = iris.data
y = iris.target  # already numeric: 0,1,2

print("X shape:", X.shape, "| y shape:", y.shape)
print("Classes:", iris.target_names)



[Keras - Multiclass Classification (Iris)]
X shape: (150, 4) | y shape: (150,)
Classes: ['setosa' 'versicolor' 'virginica']


In [ ]:
# ----------------------------
# C2) Split + Scale
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

num_classes = len(np.unique(y_train))


In [ ]:
# ----------------------------
# C3) Define Model
# ----------------------------
# For multiclass:
# - output layer has number of neurons = number of classes
# - activation = softmax to output class probabilities summing to 1
keras_multi = Sequential([
    Dense(16, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    Dense(16, activation="relu"),
    Dense(num_classes, activation="softmax")
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# ----------------------------
# C4) Compile
# ----------------------------
# Because labels are integers (0,1,2), we use sparse_categorical_crossentropy.
keras_multi.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# ----------------------------
# C5) Fit
# ----------------------------
history = keras_multi.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=16,
    verbose=0,
    callbacks=[EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)]
)

In [ ]:
# ----------------------------
# C6) Evaluate
# ----------------------------
loss, acc = keras_multi.evaluate(X_test_scaled, y_test, verbose=0)
print("Keras Multiclass Test Loss:", round(loss, 4))
print("Keras Multiclass Test Accuracy:", round(acc, 4))


Keras Multiclass Test Loss: 0.0712
Keras Multiclass Test Accuracy: 0.9667


In [ ]:
# ----------------------------
# C7) Predict Class
# ----------------------------
# model.predict gives probabilities; argmax gives predicted class index.
probs = keras_multi.predict(X_test_scaled[:5], verbose=0)
pred_class = np.argmax(probs, axis=1)

print("\nFirst 5 probability vectors:\n", np.round(probs, 4))
print("Predicted classes:", pred_class)
print("True classes     :", y_test[:5])
print("Predicted names  :", iris.target_names[pred_class])



First 5 probability vectors:
 [[9.999e-01 1.000e-04 0.000e+00]
 [3.000e-04 1.886e-01 8.111e-01]
 [3.320e-02 9.639e-01 2.800e-03]
 [3.200e-03 9.961e-01 7.000e-04]
 [9.999e-01 1.000e-04 0.000e+00]]
Predicted classes: [0 2 1 1 0]
True classes     : [0 2 1 1 0]
Predicted names  : ['setosa' 'virginica' 'versicolor' 'versicolor' 'setosa']


In [ ]:
# ============================================================
# SECTION D: Regression (Scikit-learn MLPRegressor + Keras)
# Dataset: California Housing
# ============================================================

print("\n[Regression - California Housing]")


[Regression - California Housing]


In [ ]:
# ----------------------------
# D1) Load Data
# ----------------------------
housing = fetch_california_housing()
X = housing.data
y = housing.target  # Median house value (in 100,000s)

print("X shape:", X.shape, "| y shape:", y.shape)
print("Feature names:", housing.feature_names[:5], "...")


X shape: (20640, 8) | y shape: (20640,)
Feature names: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population'] ...


In [ ]:
# ----------------------------
# D2) Split + Scale
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


In [ ]:

# ----------------------------
# D3) Scikit-learn MLPRegressor
# ----------------------------
reg = MLPRegressor(
    hidden_layer_sizes=(64, 32),
    activation="relu",
    solver="adam",
    alpha=1e-4,
    max_iter=200,
    random_state=42
)
reg.fit(X_train_scaled, y_train)

y_pred = reg.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("\nScikit-learn MLPRegressor RMSE:", round(rmse, 4))


Scikit-learn MLPRegressor RMSE: 0.5237


In [ ]:
# D4) Keras Regression Model
# ----------------------------
# For regression:
# - output layer usually has 1 neuron with linear activation (default)
keras_reg = Sequential([
    Dense(64, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)  # linear output for continuous prediction
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Loss for regression: mse
keras_reg.compile(optimizer="adam", loss="mse")

history = keras_reg.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    verbose=0,
    callbacks=[EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)]
)

test_mse = keras_reg.evaluate(X_test_scaled, y_test, verbose=0)
test_rmse = np.sqrt(test_mse)

print("Keras Regression RMSE:", round(test_rmse, 4))


Keras Regression RMSE: 0.5325


In [ ]:
# ----------------------------
# D5) Predict Example
# ----------------------------
example_row = X_test_scaled[0:1]  # one row
pred_value = keras_reg.predict(example_row, verbose=0).item()

print("\nExample prediction (Keras):", round(pred_value, 4))
print("True value:", round(y_test[0], 4))



Example prediction (Keras): 0.5172
True value: 0.477


In [ ]:
# ============================================================
# SECTION E: Quick Teaching Recap (Printed)
# ============================================================
print("\n" + "="*60)
print("TEACHING RECAP")
print("- Scikit-learn: MLPClassifier / MLPRegressor = quick neural nets for tabular data")
print("- Keras lifecycle: Define -> Compile -> Fit -> Evaluate -> Predict")
print("- Binary: sigmoid + binary_crossentropy")
print("- Multiclass: softmax + sparse_categorical_crossentropy")
print("- Regression: linear output + mse")
print("="*60)


TEACHING RECAP
- Scikit-learn: MLPClassifier / MLPRegressor = quick neural nets for tabular data
- Keras lifecycle: Define -> Compile -> Fit -> Evaluate -> Predict
- Binary: sigmoid + binary_crossentropy
- Multiclass: softmax + sparse_categorical_crossentropy
- Regression: linear output + mse
